# Lab 4.1: The offloading model

The objective of this lab is to introduce the concept of offloading to accelerator devices. This lab introduces target regions, accelerator devices, and memory management for the device

This tutorial is expected to run in a unix-like environment.

## Table of content:

* Offloading model
* Target directive
* Memory
    * Device memory
    * Implicit memory mapping
    * Structured memory management
    * Unstructured memory management


# The offloading model

OpenMP supports accelerator devies. These are special devices with compute capabilities that are different to the traditional CPU architectures. The most common accelerator devices is the General Purpuse Graphic Processing Unit (GPGPU or simply GPU).

Nvidia devices can be discovered using `nvidia-smi` and amd devices using `rocm-info`. Likewise, compilers like LLVM support special commands to obtain the available accelerator devices. This is the case of `llvm-omp-device-info`

In [1]:
# If you have an NVIDIA device use
!nvidia-smi

Tue Jun 21 20:11:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 470.74       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P1000        Off  | 00000000:01:00.0 Off |                  N/A |
| 27%   44C    P0    N/A /  N/A |      0MiB /  4040MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# If you have an AMD device use
!rocm-info

In [17]:
# If you're using llvm use
!llvm-omp-device-info

Device (0):
    This is a generic-elf-64bit device

Device (1):
    This is a generic-elf-64bit device

Device (2):
    This is a generic-elf-64bit device

Device (3):
    This is a generic-elf-64bit device

Device (4):
    CUDA Driver Version: 		11040 
    CUDA Device Number: 		0 
    Device Name: 			Quadro P1000 
    Global Memory Size: 		4236312576 bytes 
    Number of Multiprocessors: 		5 
    Concurrent Copy and Execution: 	Yes 
    Total Constant Memory: 		65536 bytes
    Max Shared Memory per Block: 	49152 bytes 
    Registers per Block: 		65536 
    Warp Size: 				32 Threads 
    Maximum Threads per Block: 		1024 
    Maximum Block Dimensions: 		1024, 1024, 64 
    Maximum Grid Dimensions: 		2147483647 x 65535 x 65535 
    Maximum Memory Pitch: 		2147483647 bytes 
    Texture Alignment: 			512 bytes 
    Clock Rate: 			1480500 kHz
    Execution Timeout: 			No 
    Integrated Device: 			No 
    Can Map Host Memory: 		Yes 
    Compute Mode: 			DEFAULT 
    Concurrent Kernels: 		Y

# Target directive
The target directive allows a programmer to use accelerator devices, as long as the device is supported by the implementation. For the remaining of this tutorial we will use LLVM with the Clang front end and it will assume that the device is an NVIDIA device. Other compilers that support OpenMP offloading can be found in the [compilers section of the OpenMP website](https://www.openmp.org/resources/openmp-compilers-tools/).

## Our first target program
One of the simplest target programs that one can write is the following 

```C
#include <stdio.h>
#include <omp.h>

int main() {
    int a[1] = {0};
    #pragma omp target
    {
        a[0] = omp_is_initial_device();
    }
    printf("Code executed in the %s\n",a[0] ? "Host":"Device");
    return 0;
}
```

In [21]:
# Building this code
!clang -fopenmp -fopenmp-targets=nvptx64 C/simple_target.c -o C/simple_target.exe

In [19]:
# Running the code on the host
!OMP_TARGET_OFFLOAD=disabled C/./simple_target.exe

Code executed in the Host


In [22]:
# Running the code on the host
!OMP_TARGET_OFFLOAD=mandatory C/./simple_target.exe

Code executed in the Device


Open and play with this code in [simple_multiple_devices](C/simple_target.c).

If the above code fails, there's a problem with your compiler. Please make sure to fix your running environment before continuing this lab.
The `OMP_TARGET_OFFLOAD` is an environment variable that allows users to specify if device offloading is enabled, disabled, or mandatory.

In this example we are cheching where the code was executed by using the API function `omp_is_initial_device()`. This API function returns true if the execution environment is the same device that innitiated the target region, i.e. the host. The `omp target` directive does not expose any parallelism per-se. Instead, it tells the compiler that the enclosed region is meant to execute in the device. The compiler will generate different versions of the same code, for different possible devices. Notice that target regions are not necessarily executed on the accelerator device (i.e. GPU in our case). It is still possible to control if target regions are executed in the host as well. This means that this code has a host version and a device version.

```
Note: An array of only 1 possition is used instead of a scalar to avoid having to keep the code simple. As we will learn later on, arrays have a default mapping of tofrom, while scalars are firstprivate. Allowing to avoid using explicit data mapping. 
```

## Controling target regions
Some important clauses to mention for the target construct are:
* `if(condition)`: Allows code to be conditionally executed in the device.
* `device(device_num)`: When multiple devices are available, allows to select the different devices
* `nowait`: Enables asynchronous execution of code. More on this on a future lab.

Take for example the following code

```C
#include <stdio.h>
#include <omp.h>

int main() {
    int num_dev = omp_get_num_devices();
    num_dev = num_dev > 10 ? 10 : num_dev;
    int device_num[11];
    int i;
    // Iterate over each available device and execute code
    // If i == omp_get_num_devices(), execute on the host.
    for (i = 0; i <= num_dev; i++) {
        #pragma omp target device(i) if(i != num_dev)
        {
            device_num[i] = omp_get_device_num();
        }
    }

    // Print which device was used for each region.
    for (i = 0; i <= num_dev; i++)
        printf("Code executed for i = %d in device %d\n", i,device_num[i]);
    return 0;
}
```

In [1]:
# Building the example
!clang -fopenmp -fopenmp-targets=nvptx64 C/simple_multiple_devices.c -o C/simple_multiple_devices.exe

In [2]:
# Running the example
!C/./simple_multiple_devices.exe

Code executed for i = 0 in device 0 that is not the initial device
Code executed for i = 1 in device 1 that is the initial device


Open and play with this code in [simple_multiple_devices](C/simple_multiple_devices.c).

## Data mapping
Often case, devices feature a memory and corresponding address space that is independent from the host. While OpenMP supports `unified address space` and `unified shared memory`, learning to manage data between host and device is really important for application performance. OpenMP refers to **"mapping"** as the process of moving variables from the host to the device and from the device to the host, allocating variables and deallocating variables.

The following example will show that a variable within a target region has a different address value, as long as this code is not executed with unified shared memory support.

```C
#include <stdio.h>
#include <omp.h>

int main() {
    int a[1];

    printf("Addres of a in host = %lx", (unsigned long)a);
    
    #pragma omp target
    {
        printf("Addres of a in device = %lx", (unsigned long)a);
    }
    return 0;
}
```

In [33]:
# Building this code
!clang -fopenmp -fopenmp-targets=nvptx64 C/different_addresses.c -o C/different_addresses.exe

In [34]:
# Running the code
!C/./different_addresses.exe

Addres of a in host = 7fffbc7cafa8
Addres of a in device = 7fb985a00000


What is important to notice here is that the address of `a` is different in the Host and the Device. If you want to play with this code open [different_addresses.c](C/different_addresses.c)

Mapping can be of the form:
* `to`: From host to device.
* `from`: From device to host.
* `tofrom`: Both from host to device at the beginning of the region, and from device to host at the end of the region.
* `alloc`: Only allocate memory, but do not copy values over to the device
* `delete`: Used with unstructured data mapping (see below). Delete a variable in the device

## Implicit data mapping

So far all the example codes we have shown do not use the `map()` clause. Whenever a variable is referenced inside of a target region, and this variable is not in a `map()` clause, it is said that this variable is implicitely mapped.

Different variable types have different implicit data mappings. While the complete list of rules can be found in the [specification document](https://www.openmp.org/specifications/), here is a set of rules of thumb that developers should follow. 

### Scalar variables
Variables that use scalar data types such as `int`, `double`, `float`, etc are mapped as `firstprivate` by default. Therefore, whenever these variables are implicitely mapped, and modified on the devices, they are not copied back to the host. The following example shows this behavior.

```C
int a = 10;
#pragma omp target // implicit firsprivate(a)
{
    printf("a = %d\n",a);
    a = 20;
}
printf("a = %d\n",a);
```

In [38]:
# building
!clang -fopenmp -fopenmp-targets=nvptx64 C/implicit_map_scalar.c -o C/implicit_map_scalar.exe

In [39]:
# Running
!C/./implicit_map_scalar.exe

a = 10
a = 10


Although a is modified in the device, the value `20` is not seen by the host after the target region. Variable `a` is firstprivatized. 

Use the file [implicit_map_scalar.c](C/implicit_map_scalar.c) to modify and play with the above code.

### Non scalar types (classes and structs)
User defined types in C and C++ are mapped as `map(tofrom:...)` by default. This means that in stand alone target regions, these variables are copied over to the device at the beginning of the region, and from the device to the host at the end of the region.

```
Note: Be aware that mapping copies contiguous memory regions. Therefore, no deep copy is performed by default. In order to support deep copy the user must either create a "declare mapper" or specify mapping of the different attribute memebers. These cases are ignored for now as it is beyond the purpose of this section
```
The following is an example of a default mapping of an struvct

```C
typedef struct myS{
    int a;
    double *b;
}myS_t;

int main() {
    myS_t myStruct = {1, NULL};

    myStruct.b = (double *)malloc(sizeof(double));
    *(myStruct.b) = 11.1;

    printf("Host {%d, %lx}\n", myStruct.a, myStruct.b);

    #pragma omp target // implicit map(tofrom:myStruct). Not implicit map(tofrom:myStruct.b[0:1])
    {
        myStruct.a = 10;
        // printf("%f\n", myStruct.b); error since b is not deep copied
        printf("Device {%d, %lx}\n", myStruct.a, myStruct.b);
    }

    printf("Host {%d, %lx}\n", myStruct.a, myStruct.b);
    return 0;
}
```

In [43]:
# building
!clang -fopenmp -fopenmp-targets=nvptx64 C/implicit_map_struct.c -o C/implicit_map_struct.exe

In [44]:
# Running
!C/./implicit_map_struct.exe

Host {1, 558eb743f030}
Device {10, 558eb743f030}
Host {10, 558eb743f030}


Notice how the Device prints the same address for `myStruct.b` in the host and the device. However, when running on a non unified memory address space, this is not a valid pointer in the device. Attempting to reference it could lead to a segmentation fault. 

Play with this code in [implicit_map_struct.c](C/implicit_map_struct.c).

### Arrays (not pointers)

Arrays for which the compiler can determine the size are also mapped as `map(tofrom:...)`.

```C

int main() {
    int A[] = {1,2,3};

    printf("Host {%d, %d, %d}\n", A[0],A[1],A[2]);

    #pragma omp target // implicit map(tofrom:A[0:3])
    {
        printf("Device {%d, %d, %d}\n", A[0],A[1],A[2]);
        A[0]++; A[1]++; A[2]++;
    }

    printf("Host {%d, %d, %d}\n", A[0],A[1],A[2]);
    return 0;
}
```

In [45]:
# building
!clang -fopenmp -fopenmp-targets=nvptx64 C/implicit_map_arrays.c -o C/implicit_map_arrays.exe

In [46]:
# Running
!C/./implicit_map_arrays.exe

Host {1, 2, 3}
Device {1, 2, 3}
Host {2, 3, 4}


To play with this code go to [implicit_map_arrays.c](C/implicit_map_arrays.c)

### Pointers

Pointers are a special case. Pointers are also mapped `map(tofrom:...)` by default. However, since it is not possible to determine how many elements are pointed by a pointer, the compiler cannot detemrine the size of the map. Pointers are therefore mapped as `tofrom:ptr[0:0]` where `[0:0]` means, starting from the 0 possition, copy 0 elements. This is confusing at first, but it allows the compiler to perform pointer translation when the array has been previously mapped to the device (e.g. using structured or unstructured data mapping as we will see later on). 

```C

int main() {
    int *A = (int*) malloc(3*sizeof(int));
    A[0] = 1; A[1] = 2; A[2] = 3;

    printf("Host A = {%d, %d, %d}\n", A[0],A[1],A[2]);

    #pragma omp target // implicit map(tofrom:A[0:0])
    {
        // printf("Device A = {%d, %d, %d}\n", A[0],A[1],A[2]); Error since pointer is not mapped. 
        // A[0]++; A[1]++; A[2]++; Error since pointer is not mapped
        printf("Cannot access A[]\n");
    }

    #pragma omp target data map(tofrom: A[0:3])
    {
        #pragma omp target // implicit mapping map(tofrom:A[0:0])
        {
            // A is automatically translated to a previously mapped location
            printf("Device A = {%d, %d, %d}\n", A[0],A[1],A[2]); 
            A[0]++; A[1]++; A[2]++;
        }
    }

    printf("Host A = {%d, %d, %d}\n", A[0],A[1],A[2]);
    return 0;
}
```

In [49]:
# building
!clang -fopenmp -fopenmp-targets=nvptx64 C/implicit_map_pointers.c -o C/implicit_map_pointers.exe

In [50]:
# Running
!C/./implicit_map_pointers.exe

Host A = {1, 2, 3}
Cannot access A[]
Device A = {1, 2, 3}
Host A = {2, 3, 4}


The first region uses a default map of A[0:0]. Therefore, no allocation has been made in the target device. Attempting to access A will lead to undefined behavior (likely a segmentation fault). Default mapping of pointers is useful when variables are already in the device, like in the second part of the example. In this case no data movement is made when entering the `target` region, but A inside the target region points to the already mapped variable from the `target data` region. Bear with us a little longer to understand `target data`. 

To play with this code go to [implicit_map_pointers.c](C/implicit_map_pointers.c)

## Explicit data mapping

In order to control how variables are mapped to/from the device, the `map()` clause is used. In a target region each variable can be mapped as `to`, `from`, `tofrom`, or `alloc`. To use different mappings for different variables, multiple `map()` clauses can be specified. Likewise, a single `map()` clause can contain multiple variables. Here are some examples:

* `map(A)`: Maps variable A `tofrom`.
* `map(tofrom: B)`: Maps variable B `tofrom`.
* `map(alloc: C, D)`: Maps variable C and D alloc.
* `map(from: E) map(to: F)`: Maps variable E as `from` and F as `to`.

Let's take a look at these different mappings in action in the following code. Let's ignore the array section notation (i.e. `[:]` and `[0:10]`) as this will be discussed shortly. Can you guess what would be the values of the variables before and after the target region?

```C
    typedef struct myStr_s{
        char a[10];
        int b;
    }myStr;
    ...
    
    int x = 3;
    int y[3] = {1,2,3};
    int *z = (int*)malloc(3 * sizeof(int));
    z[0] = 1; z[1] = 2; z[2] = 3;
    myStr s = {"aaa",0};
    myStr sa[3] = {{"hola", 1},{"chao", 2},{"hello", 3}};

    #pragma omp target          \
            map (alloc: y[:])   \
            map (to: z[0:3])    \
            map (from: sa)
    {
        x = 44;
        y[0] = 1; y[1] = 44; y[2] = 2;
        z[0] = 99; z[1] = 44; z[2] = 2;
        strcpy(sa[0].a, "hey");
        sa[0].b = 3;
        strcpy(sa[1].a, "you");
        sa[1].b = 4;
        strcpy(sa[2].a, "there");
        sa[2].b = 5;
        strcpy(s.a, "hi");
        s.b = 42;
    }

```

In [62]:
# building
!clang -fopenmp -fopenmp-targets=nvptx64 C/explicit_map.c -o C/explicit_map.exe

In [63]:
# running
!C/./explicit_map.exe

In host: 
x = 3,
y = {1, 2, 3},
z = {1, 2, 3},
s = {aaa, 0},
sa = [{hola, 1},{chao, 2},{hello, 3}]
In device:
x = 3,
y = {0, 0, 0},
z = {1, 2, 3},
s = {aaa, 0},
sa = [{, 0},{, 0},{, 0}]
modifying variables
In device modified:
x = 44,
y = {1, 44, 2},
z = {99, 44, 2},
s = {k, 42},
sa = [{h, 3},{y, 4},{t, 5}]
In host: 
x = 3,
y = {1, 2, 3},
z = {1, 2, 3},
s = {k, 42},
sa = [{h, 3},{y, 4},{t, 5}]


If you want to play with this code open [explicit_map.c](C/explicit_map.c)
The following rules apply:

* `x` was mapped as `firsprivate(x)` because of the implicit mapping behavior
* `y` was only allocated in the device, but its content only lived in the device. Data was never sent from host to device or the other way around.
* `z` was allocated in the device and initialized with the host value. However, modifications in the device were not sent over to the host, because data is not copied back from the device to the host.
* `s` was implicitly mapped as `map(tofrom:s)`, therefore, it is allocated in the device, initialized with the host value, and then the host is updated with the device at the end of the target region.
* `sa` was allocated in the device but not initialized. Therefore the values that are seeing when printing it the first time inside the device. Values are transfered to the host from the device at the end of the target region, updating the host values. 

It is important to notice that the char array with static size. In memory these elements are consecutive, and proceded by the integer value of the struct. Memory looks as follows. 

`| char[0] | char[1] | char[2] | ,,, | char[9] | int |` 

Mapping copies consecutive memory regions, making it possible to have a copy of the char array. If s.a were a pointer instead of an array, the value would have not been copied.

# Array sections

When mapping arrays, it is possible to only map sections of the array. To do this, OpenMP uses an array section notation with the following syntax:

`base_ptr[lower_bound : length : stride]`

Currently the LLVM C compiler (*clang*) only supports `lower_bound` and `lenght`.

Array sections are useful to map data to different devices, as well as to reduce size of memory copies. Take for example the following code:

```C
int A[100];
for (int i = 0; i < 50; i++)
   A[i] = i;

#pragma omp target map(to:A[0:50]) map(from:A[50:50])
{
    for (int i = 50; i < 100; i++) {
        A[i] = A[i-50];
    }
}
```

In this example, only the first 50 elements of the array A are copied over to the device, and the last 50 elements are copied back from the device.

In [25]:
# building
!clang -fopenmp -fopenmp-targets=nvptx64 C/map_array_sections.c -o C/map_array_sections.exe

In [26]:
# running
!C/./map_array_sections.exe

A = [ 0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 ]
B = [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ]
A = [ 0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  

If you want to play with this code open [map_array_sections.c](C/map_array_sections.c)

## Structured data mapping

Structured data mapping refers to a `target data` region, enclosed by a pair of brackets that mark a beginning and end of a data environment. Structured mapping uses the `target data` which is different to the `target` directive. The `target data` directive **do not offload code to the device**, rather, it creates a data environment in the device, allocating variables and moving data between host and device according to the `map()` clauses.

Structured data mapping allows to easily understand the scope of variables in the device. However, it can only be used within the same block of code. Therefore, the beginning and end regions cannot exist in different functions.

Here's an example:

```C
int A[100];

for (int i = 0; i < 100; i++)
       A[i] = i;
#pragma omp target data map(tofrom: A)
{
    for (int i = 0; i < 100; i++) // EXECUTED IN THE HOST
       A[i] = 0;
}
```

This example highlights two aspects of the `target data` region. First, the code inside of the `target data` region (unless it is a `target` region itself) is executed in the host. Second, at the end of the target region, data is copied back to the host, when variables are mapped `from`. Therefore, data in the host will be overwritten. 

What do you think is the value of `A[]` after the end of the `target data` region?

In [28]:
# building
!clang -fopenmp -fopenmp-targets=nvptx64 C/target_data_simple.c -o C/target_data_simple.exe

In [29]:
# running
!C/./target_data_simple.exe

A = [ 0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 ]
A = [ 0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 ]


The answer is that the modification of A inside of the target data region `A[i] = 0;` is not visible outside of the `target data` region. Because the end of the `target data` region overwrites A again with the value sent to the device in the first place. 

Play with this code in [target_data_simple.c](C/target_data_simple.c)

Now, let's take a look at the effect of using `target data` and how it can help us avoid moving data around. An important rule of thumb when programming accelerator devices is that data movements are often expensive. It is necessary to avoid them as much as possible, and only leave those operations that are absolutely necessary.

Take for example the following code. This is often a really common pattern in many applications.

```C
int A[N];
for (int step = 0; step < STEPS; step++)
   #pragma omp target
   {
      foo(A);
   }
```

Each time a new step is executed, the target region will move data from host to the device, and the other way around. These operations add to a considerable overhead.

By using `target data` it is possible to create a data environment that remains between multiple executions of the target region. 

```C
int A[N];
#pragma target data map(tofrom:A)
{
for (int step = 0; step < STEPS; step++)
   #pragma omp target
   {
      foo(A);
   }
}
```

The OpenMP runtime will only move data at the beginning and at the end of the `target data` region. Let's compare execution times for both.

In [38]:
# Compiling
!clang -fopenmp -fopenmp-targets=nvptx64 C/target_data_simulation.c -o C/target_data_simulation.exe

In [39]:
# Running
!C/./target_data_simulation.exe

Exec time without target data = 2.367565
Exec time with target data = 0.485867


Consider that in both cases, the `target` region is doing exactly the same work, moving data back and forth increases the execution time considerably.

```
Note: We have not exposed parallelism in the GPU so far, so the code from the example above is a really bad example on how to use GPUs, but it emphasizes the importance of data movements for computation. 
```

Play with this code opening the file [target_data_simulation.c](C/target_data_simulation.c)